**NLP-Powered SQL Helper**

In this notebook I have created a project of an NLP model that helps users construct SQL commands based on natural language queries. For example, a user might type: "Show me all customers from United Kingdom" and the code will generate this SQL command: `SELECT * FROM customers WHERE state = 'United Kingdom'; `

For extraction, I used [spaCY](https://spacy.io/), a popular NLP library that is efficient, easy to use, and provides pre-trained models, making it suitable for tasks like entity recognition and keyword extraction. 

**Key Features**:

- Accept user inputs in natural language.

- Extract important entities such as tables, fields, conditions, etc.

- Generate SQL commands based on parsed input.

In [1]:
# Install spacy if not installed
#!pip install spacy --quiet

## Creating Database:

I will be using an SQLite database, because it is lightweight, easy to set up, and doesn't require a server setup. 

Below are the steps I followed to create and populate my database:

1. Downloaded the dataset from a public [repository](https://archive.ics.uci.edu/datasets?skip=0&take=10&sort=desc&orderBy=NumHits&search=Online+Retail), and loaded it to a pandas dataframe.
2. Created a connection to an SQLite database, and created a table.
3. Inserted the data into the table, verified the insertation by executing a simple SQL command into the database. 

In [2]:
import sqlite3
import pandas as pd
import urllib.request
import zipfile

# Downloading a public dataset and loading the dataset using pandas
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
file_path = "Online_Retail.xlsx"
urllib.request.urlretrieve(url, file_path)

df = pd.read_excel(file_path)
df.head(5)  # Viewing the first 5 rows in table format

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Creating a connection to SQLite database and a table
conn = sqlite3.connect('retail_data.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS retail (
        InvoiceNo TEXT,
        StockCode TEXT,
        Description TEXT,
        Quantity INTEGER,
        InvoiceDate TEXT,
        UnitPrice REAL,
        CustomerID TEXT,
        Country TEXT
    )
''')

# Inserting data into the table and verify data insertion
df.to_sql('retail', conn, if_exists='replace', index=False)

cursor.execute("SELECT * FROM retail LIMIT 5")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()

('536365', '85123A', 'WHITE HANGING HEART T-LIGHT HOLDER', 6, '2010-12-01 08:26:00', 2.55, 17850.0, 'United Kingdom')
('536365', '71053', 'WHITE METAL LANTERN', 6, '2010-12-01 08:26:00', 3.39, 17850.0, 'United Kingdom')
('536365', '84406B', 'CREAM CUPID HEARTS COAT HANGER', 8, '2010-12-01 08:26:00', 2.75, 17850.0, 'United Kingdom')
('536365', '84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE', 6, '2010-12-01 08:26:00', 3.39, 17850.0, 'United Kingdom')
('536365', '84029E', 'RED WOOLLY HOTTIE WHITE HEART.', 6, '2010-12-01 08:26:00', 3.39, 17850.0, 'United Kingdom')


## Text extraction using NLP

**How I'm using spaCy in this context?**

- Let's say I want to write an SQL command for the table created above that will display all the customers from the United Kingdom who made purchases from the online retail store.

- First, I load an English NLP model from spaCy and define my query as ```user_query = Show me all customers from United Kingdom```. I then process the query using the loaded model: ```doc = nlp(user_query)```.  

- Next, I extract entities and keywords from the processed query. Below, I've shared an example of the code I used to extract the data and explained what each output represents:

    - ```Token: Show, POS: VERB, Dependency: ROOT```: "Show" is identified as a verb (POS: Part of Speech), and it's the root of the sentence, meaning it's the main action.
    - ```Token: me, POS: PRON, Dependency: dative```: "Me" is a pronoun, and it has a "dative" dependency, meaning it’s the indirect object of the verb.
    - ```Token: all, POS: DET, Dependency: det```: "All" is a determiner, specifying the noun "customers".
    - ```Token: customers, POS: NOUN, Dependency: dobj```: "Customers" is a noun and is the direct object (dobj) of the action ("Show").
    - ```Token: from, POS: ADP, Dependency: prep```: "From" is an adposition (typically a preposition), indicating a relation between "customers" and a location.
    - ```Token: United Kingdom, POS: PROPN, Dependency: pobj```: "United Kingdom" is a proper noun and is the object of the preposition "from".

- I can also extract specific entities, such as a [named entity](https://spacy.io/usage/spacy-101#annotations-ner), which refers to “real-world objects” that are assigned a name, for example, places or countries. The results will be as follows:



  - ```Entity: United Kingdom, Label: GPE```: "United Kingdom" is recognized as a Geopolitical Entity (GPE), which means it represents a location such as a country, city, or state.


Below is an example of the code I executed for this scenario and the corresponding results:

In [4]:
import spacy

# Loading the model
nlp = spacy.load('en_core_web_sm')

# Defining and processing the user_query
user_query = "Show me all customers from United Kingdom"
doc = nlp(user_query)

# Extracting the entities and keywords
for token in doc:
    print(f"Token: {token.text}, POS: {token.pos_}, Dependency: {token.dep_}")

# Extracting specific entities like nouns
for ent in doc.ents:
    print(f"\nEntity: {ent.text}, Label: {ent.label_}")

Token: Show, POS: VERB, Dependency: ROOT
Token: me, POS: PRON, Dependency: dative
Token: all, POS: DET, Dependency: det
Token: customers, POS: NOUN, Dependency: dobj
Token: from, POS: ADP, Dependency: prep
Token: United, POS: PROPN, Dependency: compound
Token: Kingdom, POS: PROPN, Dependency: pobj

Entity: United Kingdom, Label: GPE


#### Code
Below is a function that will construct and return an SQL command based on a user_query with optional conditions:

In [5]:
import spacy

def generate_sql_query(user_query):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(user_query)

    table = "retail"
    columns = []
    conditions = []

    field_mapping = {
        "customer": "CustomerID",
        "date": "Date",
        "description": "Description",
        "item": "Quantity",
        "invoice": "InvoiceNo"
    }

    # Extracting entities and keywords as conditions
    for ent in doc.ents:
        if ent.label_ == "GPE":  # Geographical location 
            conditions.append(f"Country = '{ent.text}'")
        elif ent.label_ == "CARDINAL":  # Extracts quantity
            conditions.append(f"Quantity >= {ent.text}")
        elif ent.label_ == "PERSON":  # Extracts customerID
            conditions.append(f"CustomerID = '{ent.text}'")
        elif ent.label_ == "DATE":  # Extracts dates, but since spaCy recognises InvoiceNo as Date, so it is handled separately
            if "InvoiceNo" in user_query or ent.text.isdigit():
                conditions.append(f"InvoiceNo = '{ent.text}'")
            else:
                conditions.append(f"Date = '{ent.text}'")
    
    # Extracting tokens and checking whether the token is an adjective describing/modifying a noun

    for token in doc:
        if token.dep_ == "amod" and token.head.pos_ == "NOUN":
            columns.append(token.head.text)
        elif token.text.lower() in field_mapping:
            columns.append(field_mapping[token.text.lower()])
        elif token.dep_ == "nummod" and token.head.text.lower() in field_mapping:
            conditions.append(f"{field_mapping[token.head.text.lower()]} = {token.text}")
        elif token.dep_ == "pobj" and token.head.text.lower() == "description":
            conditions.append(f"Description LIKE '%{token.text}%'")
        elif token.dep_ == "pobj" and token.head.text.lower() == "customer":
            conditions.append(f"CustomerID = '{token.text}'")

    # Constructing the SQL query
    columns_str = ', '.join(columns) if columns else '*'
    query = f"SELECT {columns_str} FROM {table}"
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    query += " LIMIT 5"  # Setting the limit to 5, for demo purposes

    return query

Below I have tested the function with my database:

In [6]:
# Example usage with SQLite database
def execute_sql_query(user_query):

    sql_query = generate_sql_query(user_query)
    print(f"Generated SQL Query: {sql_query}\n")

    # Connect to the SQLite database
    conn = sqlite3.connect('retail_data.db')
    cursor = conn.cursor()

    # Execute the generated SQL query
    try:
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        for row in rows:
            print(row)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

    # Close the connection
    conn.close()

Testing the above conditions

In [7]:
user_query_1 = "Show customers from United Kingdom who bought 50 or more items"
execute_sql_query(user_query_1)

Generated SQL Query: SELECT * FROM retail WHERE Country = 'United Kingdom' AND Quantity >= 50 LIMIT 5

('536371', '22086', "PAPER CHAIN KIT 50'S CHRISTMAS ", 80, '2010-12-01 09:00:00', 2.55, 13748.0, 'United Kingdom')
('536376', '21733', 'RED HANGING HEART T-LIGHT HOLDER', 64, '2010-12-01 09:32:00', 2.55, 15291.0, 'United Kingdom')
('536378', '21212', 'PACK OF 72 RETROSPOT CAKE CASES', 120, '2010-12-01 09:37:00', 0.42, 14688.0, 'United Kingdom')
('536378', '85071B', 'RED CHARLIE+LOLA PERSONAL DOORSIGN', 96, '2010-12-01 09:37:00', 0.38, 14688.0, 'United Kingdom')
('536382', '22381', 'TOY TIDY PINK POLKADOT', 50, '2010-12-01 09:45:00', 1.85, 16098.0, 'United Kingdom')


In [8]:
user_query_2 = "Show the description where InvoiceNo is 536365"
execute_sql_query(user_query_2)

Generated SQL Query: SELECT Description FROM retail WHERE InvoiceNo = '536365' LIMIT 5

('WHITE HANGING HEART T-LIGHT HOLDER',)
('WHITE METAL LANTERN',)
('CREAM CUPID HEARTS COAT HANGER',)
('KNITTED UNION FLAG HOT WATER BOTTLE',)
('RED WOOLLY HOTTIE WHITE HEART.',)


## Conclusion

Using a pretrained spaCy model for an NLP task, such as generating SQL queries from user input works well for basic tasks like identifying countries. However, as shown in this example:

```
user_query = "Show items on invoice 536365 "
-----------

> Token: Show, POS: VERB, Dependency: ROOT
Token: items, POS: NOUN, Dependency: dobj
Token: on, POS: ADP, Dependency: prep
Token: invoice, POS: NOUN, Dependency: pobj
Token: 536365, POS: NUM, Dependency: nummod

Entity: 536365, Label: DATE
```

There are limitations. In this case, the model mistakenly treated the InvoiceNo as a DATE entity instead of a CARDINAL. This happens because spaCy struggles to understand the context in complex queries and can misinterpret numbers. As such, using a pretrained spaCy model for such use-cases is not efficient. 

Lastly, I will share the transformer-based code soon, which I will use to try and solve more complex queries. 